In [1]:
import os
import sys 
import configparser
import pandas as pd 
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from matplotlib import cycler
from os.path import join as oj
from torch.utils.data import TensorDataset, DataLoader
import pickle as pkl
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sys.path.insert(0, "..")
import training.models
from config.model_config import model_config
import seaborn as sns
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.2f' % x)
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
device = torch.device("cpu")

# Loading the dataset

In [7]:
import os 
import sys 
src_path = "/Users/paolovincenzofreieslebendeblasio/battery-life-prediction/src"
sys.path.append(src_path)
import data_loader as dl 
from config_data import data_config as config 

data_dict = dl.load_data_all_channels(config["abs_data_path"])
#Path to the .hd5 file containing cycling data measured with the arbin system 

x, y, c, var  = dl.get_capacity_input(data_dict, num_offset=0, start_cycle = 10, stop_cycle = 100)

x_scaled  = dl.scale_x(x,y) #Scaling the QC, and cycle life 

x_preprocessed = dl.remove_outliers(x_scaled,y) #Removing outliers
x_smoothed = dl.smooth_x(x_preprocessed,y, num_points=20) #Smoothing the inputs and outputs with a moving average filter

# Splitting the data into train and test

In [1]:
seq_length = 50
train_idxs , val_idxs,test_idxs= dl.get_split(len(x), seed =11)

qc_variance_scaler = StandardScaler().fit(var[train_idxs]) 
var = qc_variance_scaler.transform(var)

augmented_data =np.hstack([c, var]) 
train_x, train_y , train_s = dl.assemble_dataset(x_preprocessed[train_idxs], y[train_idxs],  augmented_data[train_idxs],seq_len = seq_length)
_, smoothed_y , _ = dl.assemble_dataset(x_smoothed[train_idxs], y[train_idxs],  augmented_data[train_idxs],seq_len = seq_length)

min_val = .85
max_val = 1

capacity_output_scaler = MinMaxScaler((-1, 1), clip=False).fit(
    np.maximum(np.minimum(smoothed_y[:, 0:1], max_val), min_val)
)

NameError: name 'dl' is not defined

# Loading the trained models

In [9]:
results_path = model_config["save_model_path"] #Path to where we save the models 
fnames = sorted([oj(results_path, fname) for fname in os.listdir(results_path) if ".pt" in fname]) 
results = sorted([oj(results_path, fname) for fname in os.listdir(results_path) if ".pkl" in fname])[0] 
model = fnames[0] #We just pick a model from the list of models to illustrate


input_dim = train_x.shape[2]   # Number of input features (e.g. discharge capacity)
num_augment = train_s.shape[1]  # three  values of charging schedule (avg and last) plus the variance


my_models = models.Uncertain_LSTM(train_x.shape[2], 
                            train_s.shape[1] , 
                            num_hidden =  results.hidden_size, 
                            num_hidden_lstm =  results.hidden_size_lstm, 
                            seq_len= results.sequence_length[best_model_idx], 
                            n_layers =2, 
                            dropout =.0).to(device)
for i, file_name in enumerate(results.file_name):
    my_models[i].load_state_dict(torch.load(oj(model_path,file_name+".pt"),map_location=torch.device('cpu')))
    my_models[i] = my_models[i].to(device)

NameError: name 'train_x' is not defined

# Predicting the charge curve

In [11]:
cycle_dict = {}
num_samples = 10
my_quantile = .1
num_models = 1
max_steps = 3500

used_idxs =  [0,1,2]